# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *



- Collect random trajectories from the environment.
- Train an encoder using VAE on the collected trajectories.
- End-to-end training of the world model and communication module using the trained encoder.

In [ ]:
#| export
import importlib
def get_cls(module_name, class_name):
    module = importlib.import_module(module_name)
    return getattr(module, class_name)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()